In [ ]:
pip install datasets


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
import pandas as pd

In [ ]:
# Load the tokenizer and model
model_name = "Helsinki-NLP/opus-mt-mul-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/791k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
# Load your dataset (blog post articles and short headings)
train_df = pd.read_csv("/content/LABELLED_TRAIN (1).csv")
dev_df = pd.read_csv("/content/LABELLED_DEV (1).csv")


In [ ]:
# Preprocess the data
def preprocess_data(examples):
    inputs = examples["News Article"]
    targets = examples["Caption"]

    # Tokenize inputs and outputs
    model_inputs = tokenizer(inputs, max_length=300, truncation=True, padding="max_length")  # Blog post size
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=45, truncation=True, padding="max_length")    # Short headings

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:

# Convert pandas DataFrame to Hugging Face Dataset
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

In [ ]:

# Tokenize datasets
train_dataset = train_dataset.map(preprocess_data, batched=True)
dev_dataset = dev_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,  # Slightly smaller learning rate for better stability
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,  # More epochs since data is small
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=5,
    save_strategy="epoch",
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:

# Define the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer
)


<ipython-input-10-ddd15d291aae>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:

# Fine-tune the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.473500,0.558513
2,0.442200,0.537334
3,0.433200,0.531078
4,0.385200,0.530939


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[64171]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=752, training_loss=0.48079777168149646, metrics={'train_runtime': 593.4681, 'train_samples_per_second': 20.22, 'train_steps_per_second': 1.267, 'total_flos': 1627121516544000.0, 'train_loss': 0.48079777168149646, 'epoch': 4.0})

In [ ]:

# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/source.spm',
 './fine_tuned_model/target.spm',
 './fine_tuned_model/added_tokens.json')

In [ ]:
import shutil
import os

# Define the directory where the model and tokenizer are saved
model_dir = "./fine_tuned_model"

# Create a zip file from the directory
shutil.make_archive(model_dir, 'zip', model_dir)

# Check if the zip file is created
if os.path.exists(f"{model_dir}.zip"):
    print("Model saved as a zip file.")
else:
    print("Error in saving the model as a zip file.")


Model saved as a zip file.


In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c187431239c69c5d8495dfecd01395e61b6a8df3b2e743799f62a01caaa8ab46
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
Print("S")

In [ ]:
from rouge_score import rouge_scorer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from datasets import Dataset
import torch
from tqdm import tqdm  # Import tqdm for progress bar



In [ ]:

# Load the fine-tuned model and tokenizer
model_dir = "./fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)


In [ ]:

# Load dev dataset (replace with actual dev data path)
dev_df = pd.read_csv("/content/LABELLED_DEV (1).csv")  # CSV with 'article' and 'headline' columns

# Prepare the dataset for generation
def preprocess_data(examples):
    inputs = examples["News Article"]
    model_inputs = tokenizer(inputs, max_length=300, truncation=True, padding="max_length")
    return model_inputs


In [ ]:

# Convert pandas DataFrame to Hugging Face Dataset
dev_dataset = Dataset.from_pandas(dev_df)

# Tokenize dev dataset
dev_dataset = dev_dataset.map(preprocess_data, batched=True)


In [ ]:
# Generate predictions
def generate_headlines(model, tokenizer, dataset, batch_size=16):
    model.eval()  # Set model to evaluation mode
    predictions = []
    total_samples = len(dataset)

    # Loop with tqdm progress bar to monitor task progress
    for i in tqdm(range(0, total_samples, batch_size), desc="Generating Headlines", total=total_samples//batch_size):
        inputs = dataset[i:i+batch_size]['input_ids']
        attention_mask = dataset[i:i+batch_size]['attention_mask']

        # Convert inputs to tensors
        input_ids = torch.tensor(inputs).to(model.device)
        attention_mask = torch.tensor(attention_mask).to(model.device)

        # Generate the headlines
        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=45, num_beams=5, early_stopping=True)

        # Decode the outputs back to text
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(decoded_preds)
    return predictions

In [ ]:
# Generate predictions on the dev dataset
predicted_headlines = generate_headlines(model, tokenizer, dev_dataset)

# Save the predicted headlines and reference headlines to a CSV file
output_df = dev_df[['News Article', 'Caption']].copy()
output_df['predicted_headline'] = predicted_headlines


In [ ]:
# Save to CSV
output_df.to_csv('/content/generated_headlines.csv', index=False)

print("Generated headlines saved to 'generated_headlines.csv'")


In [ ]:
from rouge_score import rouge_scorer
import pandas as pd

# Load the saved generated headlines CSV file
output_df = pd.read_csv('/content/generated_headlines.csv')

# Initialize ROUGE scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store ROUGE scores
rouge_results = {'rouge1': [], 'rouge2': [], 'rougeL': []}


In [ ]:

# Calculate ROUGE scores for each prediction
for pred, ref in zip(output_df['predicted_headline'], output_df['Caption']):
    scores = rouge_scorer.score(ref, pred)
    rouge_results['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_results['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_results['rougeL'].append(scores['rougeL'].fmeasure)

# Compute average ROUGE scores
avg_rouge1 = sum(rouge_results['rouge1']) / len(rouge_results['rouge1'])
avg_rouge2 = sum(rouge_results['rouge2']) / len(rouge_results['rouge2'])
avg_rougeL = sum(rouge_results['rougeL']) / len(rouge_results['rougeL'])

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")
